# PIPEX

## Makefile

In [267]:
%%file ../pipex_working/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2023/01/30 14:27:11 by dpentlan          #+#    #+#              #
#    Updated: 2023/04/29 12:19:40 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

NAME = pipex
RED='\033[0;31m'
GREEN='\033[0;32m'
NOCOLOR='\033[0m'

# ***** Sources *****

SRCS = pipex.c \
	pipex_utils_1.c \
	pipex_utils_2.c \
	pipex_path_utils_1.c \
	pipex_fork_utils_1.c \

SRCS_DIR = srcs/

# ***** Includes *****

INCS = pipex.h

INCS_DIR = includes/

# ***** Objects *****

OBJS = $(patsubst %.c,$(OBJS_DIR)%.o,$(SRCS))

OBJS_DIR = objs/

# ***** Compile and Archiver *****

CFLAGS = -Wall -Wextra -Werror -g

CC = cc

LIBFT = -Llibft -lft

# ***** Build *****

all: $(NAME)

bonus: $(BONUS)

$(NAME): $(OBJS) $(OBJS_MAN)
	@echo -e -n ${GREEN}
	@make -C libft --silent
	@echo -e ===== MAKING PIPEX =====
	@echo -e -n ${NOCOLOR}
	@$(CC) $(CFLAGS) $(OBJS) $(LIBFT) -o $(NAME)

$(OBJS_DIR)%.o: $(SRCS_DIR)%.c
	@mkdir -p $(OBJS_DIR)
	@$(CC) -c $(CFLAGS) -I$(INCS_DIR) $< -o $@

clean:
	@make -C libft clean --silent
	@rm -f $(OBJS)
	@rm -rf $(OBJS_DIR)

fclean: clean
	@make -C libft fclean --silent
	@rm -f $(NAME)
	@rm -f gdb.txt

re: fclean all


Overwriting ../pipex_working/Makefile


## HEADERS

In [299]:
%%file ../pipex_working/includes/pipex.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.h                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:57:04 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/20 12:57:37 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef PIPEX_H
# define PIPEX_H

# include "../libft/includes/libft.h"
# include <unistd.h>
# include <sys/wait.h>
# include <error.h>

typedef struct s_pipex {
	int			fds[2];
	pid_t		pid;
	int			argc;
	char		**argv;
	char		**envp;
	char		*cmd_abspath;
	char		**cmd_args;
	char		**path_tab;
}				t_pipex;

//	pipex.c
//	int main (int argc, char **argv, char **envp);

//	pipex_utils_1.c
int		px_error(t_pipex *pipex, char *err_message);
int		ft_infile(t_pipex *pipex);
int		ft_outfile(t_pipex *pipex);
int		ft_px_init(t_pipex *pipex, int argc, char **argv, char **envp);
void	ft_free_tab(char **table);

//	pipex_utils_2.c
int		px_exec_args(t_pipex *pipex);
void	px_close_fds(t_pipex *pipex);

//	pipex_path_utils_1.c
void	px_path_tab_add_fw_slash(t_pipex *pipex);
void	px_get_path_tab(t_pipex *pipex);
void	px_build_abspath(t_pipex *pipex);
void	px_get_abspath(t_pipex *pipex);
void	px_get_execargs(t_pipex *pipex, int com_num);

//	pipex_fork_utils_1.c

#endif


Overwriting ../pipex_working/includes/pipex.h


## SOURCES

### pipex.c

In [300]:
%%file ../pipex_working/srcs/pipex.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.c                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:56:25 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/29 16:03:05 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	main(int argc, char **argv, char **envp)
{
	t_pipex	*pipex;
	t_pipex	s_pipex;

	pipex = &s_pipex;
	ft_px_init(pipex, argc, argv, envp);
	if (pipe(pipex->fds))
		px_error(pipex, "pipe");
	ft_infile(pipex);

// BIG NOTE TO ME: Access failure should not stop execution of the program, only print a note saying that the file wasnt found.
// then execution should continue

// fork loop start here //
	px_exec_args(pipex);
	pipex->pid = fork();
	if (pipex->pid < 0)
		px_error(pipex, "fork");
	else if (pipex->pid == 0)
	{
		dup2(pipex->fds[1], STDOUT_FILENO);
		close(pipex->fds[0]);
		close(pipex->fds[1]);
		execve(pipex->cmd_abspath, pipex->cmd_args, NULL);
	}
	waitpid(pipex->pid, NULL, 0);
	if (pipex->cmd_args)
	{
		ft_free_tab(pipex->cmd_args);
		pipex->cmd_args = 0;
	}
	if (pipex->cmd_abspath)
	{
		free(pipex->cmd_abspath);
		pipex->cmd_abspath = 0;
	}
// fork loop end here //

	ft_outfile(pipex);

	px_exec_args(pipex);
	pipex->pid = fork();
	if (pipex->pid < 0)
		px_error(pipex, "fork");
	if (pipex->pid == 0)
	{
		dup2(pipex->fds[0], STDIN_FILENO);
		close(pipex->fds[0]);
		close(pipex->fds[1]);
		execve(pipex->cmd_abspath, pipex->cmd_args, NULL);
	}
	close(pipex->fds[1]);
	waitpid(pipex->pid, NULL, 0);
	if (pipex->cmd_args)
	{
		ft_free_tab(pipex->cmd_args);
		pipex->cmd_args = 0;
	}
	if (pipex->cmd_abspath)
	{
		free(pipex->cmd_abspath);
		pipex->cmd_abspath = 0;
	}

	if (pipex->path_tab)
	{
		ft_free_tab(pipex->path_tab);
		pipex->path_tab = 0;
	}
	px_close_fds(pipex);
	return (0);
}


Overwriting ../pipex_working/srcs/pipex.c


### pipex_utils_1.c

In [301]:
%%file ../pipex_working/srcs/pipex_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_utils_1.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/29 17:20:54 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/09 15:31:54 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	px_error(t_pipex *pipex, char *err_message)
{
	if (ft_strnstr(err_message, "args", 4))
		ft_putstr_fd("Usage: pipex [infile] ['command'] ... [outfile]\n", 1);
	else if (ft_strnstr(err_message, "no_path", 7))
		ft_putstr_fd("path: No valid path found in envp\n", 1);
	else
		perror(err_message);
	if (pipex->cmd_abspath)
		free(pipex->cmd_abspath);
	if (pipex->cmd_args)
		ft_free_tab(pipex->cmd_args);
	if (pipex->path_tab)
		ft_free_tab(pipex->path_tab);
	px_close_fds(pipex);
	exit(EXIT_FAILURE);
}

int	ft_infile(t_pipex *pipex)
{
	int		in_file;

	in_file = 0;
	if (access(pipex->argv[1], F_OK | R_OK))
		px_error(pipex, "access");
	in_file = open(pipex->argv[1], O_RDONLY);
	if (in_file < 0)
		px_error(pipex, "open");
	dup2(in_file, STDIN_FILENO);
	close(in_file);
	return (0);
}

int	ft_outfile(t_pipex *pipex)
{
	int		out_file;
	char	*fn;

	out_file = 0;
	fn = pipex->argv[pipex->argc - 1];
	out_file = open(fn, O_CREAT | O_WRONLY | O_TRUNC, 0666);
	if (out_file < 0)
		px_error(pipex, "open");
	dup2(out_file, STDOUT_FILENO);
	close(out_file);
	return (0);
}

int	ft_px_init(t_pipex *pipex, int argc, char **argv, char **envp)
{
	if (argc < 5)
		px_error(pipex, "args");
	pipex->argc = argc;
	pipex->argv = argv;
	pipex->envp = envp;
	pipex->cmd_abspath = 0;
	pipex->cmd_args = 0;
	pipex->path_tab = 0;
	return (0);
}

void	ft_free_tab(char **table)
{
	int	i;

	i = 0;
	while (table[i])
	{
		free(table[i]);
		i++;
	}
	free(table);
	return ;
}


Overwriting ../pipex_working/srcs/pipex_utils_1.c


### pipex_utils_2.c

In [291]:
%%file ../pipex_working/srcs/pipex_utils_2.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_utils_2.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/12 18:57:46 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/12 18:57:59 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	px_exec_args(t_pipex *pipex)
{
	static int	com_num;

	if (com_num == 0)
		com_num = 2;
	else
		com_num++;
	px_get_execargs(pipex, com_num);
	px_get_abspath(pipex);
	return (0);
}

void	px_close_fds(t_pipex *pipex)
{
	close(STDIN_FILENO);
	close(STDOUT_FILENO);
	close(STDERR_FILENO);
	close(pipex->fds[0]);
	close(pipex->fds[1]);
}


Overwriting ../pipex_working/srcs/pipex_utils_2.c


### pipex_path_utils_1.c

In [296]:
%%file ../pipex_working/srcs/pipex_path_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_path_utils_1.c                               :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/12 18:58:17 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/12 18:58:25 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

void	px_path_tab_add_fw_slash(t_pipex *pipex)
{
	int		i;
	char	*buf;

	i = 0;
	buf = 0;
	while (pipex->path_tab[i])
	{
		if (pipex->path_tab[i][ft_strlen(pipex->path_tab[i]) - 1] != '/')
		{
			buf = ft_strjoin(pipex->path_tab[i], "/");
			free(pipex->path_tab[i]);
			pipex->path_tab[i] = buf;
			if (!pipex->path_tab[i])
				px_error(pipex, "malloc");
		}
		i++;
	}
}

void	px_get_path_tab(t_pipex *pipex)
{
	char	*path;
	char	**env_elem;

	path = 0;
	env_elem = pipex->envp;
	while (*env_elem)
	{
		path = ft_strnstr(*env_elem, "PATH=", 5);
		if (path)
			break ;
		env_elem++;
	}
	if (!*env_elem)
		px_error(pipex, "no_path");
	pipex->path_tab = ft_split(path + 5, ':');
	if (!pipex->path_tab)
		px_error(pipex, "malloc");
	px_path_tab_add_fw_slash(pipex);
}

void	px_build_abspath(t_pipex *pipex)
{
	int	i;

	i = 0;
	while (pipex->path_tab[i])
	{
		if (pipex->cmd_abspath)
			free(pipex->cmd_abspath);
		pipex->cmd_abspath = ft_strjoin(pipex->path_tab[i], pipex->cmd_args[0]);
		if (!pipex->cmd_abspath)
			px_error(pipex, "malloc");
		if (!access(pipex->cmd_abspath, F_OK | X_OK))
			break ;
		i++;
	}
}

void	px_get_abspath(t_pipex *pipex)
{
	if (pipex->cmd_args[0][0] == '/')
	{
		pipex->cmd_abspath = ft_strdup(pipex->cmd_args[0]);
		if (!pipex->cmd_abspath)
			px_error(pipex, "malloc");
	}
	else
	{
		if (!pipex->path_tab)
			px_get_path_tab(pipex);
		px_build_abspath(pipex);
	}
	if (access(pipex->cmd_abspath, F_OK | X_OK))
		px_error(pipex, "access");
}

void	px_get_execargs(t_pipex *pipex, int com_num)
{
	pipex->cmd_args = ft_split(pipex->argv[com_num], ' ');
	if (!pipex->cmd_args)
		px_error(pipex, "malloc");
}


Overwriting ../pipex_working/srcs/pipex_path_utils_1.c


### pipex_fork_utils_1.c

In [293]:
%%file ../pipex_working/srcs/pipex_fork_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_fork_utils_1.c                               :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/12 18:58:31 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/12 18:58:40 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"


Overwriting ../pipex_working/srcs/pipex_fork_utils_1.c


## TESTING

### Test Build

In [2]:
%%bash
cd ../pipex_working
make re

===== MAKING LIBFT =====
===== MAKING PIPEX =====


In [34]:
%%bash
cd ../pipex_working
make re > /dev/null
./pipex > /dev/null
nm -Duj pipex | grep -v "^_"
make fclean

access@GLIBC_2.2.5
close@GLIBC_2.2.5
dup2@GLIBC_2.2.5
execve@GLIBC_2.2.5
fork@GLIBC_2.2.5
free@GLIBC_2.2.5
malloc@GLIBC_2.2.5
pipe@GLIBC_2.2.5
waitpid@GLIBC_2.2.5
write@GLIBC_2.2.5


In [2]:
%%bash
# use valgrind with --track-fds=all to see if fds are closing appropriately
valgrind --track-fds=all < file1 cat | wc > file2

==86224== Memcheck, a memory error detector
==86224== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==86224== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==86224== Command: cat
==86224== 
==86224== 
==86224== FILE DESCRIPTORS: 0 open (0 std) at exit.
==86224== 
==86224== HEAP SUMMARY:
==86224==     in use at exit: 0 bytes in 0 blocks
==86224==   total heap usage: 1,051 allocs, 1,051 frees, 205,320 bytes allocated
==86224== 
==86224== All heap blocks were freed -- no leaks are possible
==86224== 
==86224== For lists of detected and suppressed errors, rerun with: -s
==86224== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)


### argument testing

In [261]:
%%bash
WORKING="../pipex_working/"
TESTING="../pipex_testing/"
cd $WORKING
rm -f valgrind.log
rm -f ${TESTING}valgrind.log
# test 4 commands that exist
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "cat -e" "/usr/bin/wc -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 cat "/usr/bin/wc -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "cat -e" wc file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 cat wc file8 &>> valgrind.log && cat file8 &>> valgrind.log
# test 16 commands that DONT exist
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "cat -e" "/usr/bin/wc -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "catlkj -e" "/usr/bin/wc -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "cat -e" "/usr/bin/wclkj -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "catlkj -e" "/usr/bin/wclkj -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 cat "/usr/bin/wc -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 catlkj "/usr/bin/wc -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 cat "/usr/bin/wclkj -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 catlkj "/usr/bin/wclkj -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "cat -e" wc file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "catlkj -e" wc file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "cat -e" wclkj file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 "catlkj -e" wclkj file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 cat wc file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 catlkj wc file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 cat wclkj file8 &>> valgrind.log && cat file8 &>> valgrind.log
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1 catlkj wclkj file8 &>> valgrind.log && cat file8 &>> valgrind.log
# test 1 valid commands but file does not exist
rm -f file8 && valgrind --track-fds=all --leak-check=full ./pipex file1lksjdlkfj "cat -e" "/usr/bin/wc -l" file8 &>> valgrind.log && cat file8 &>> valgrind.log

echo -n "Good Valgrind Runs: Memory: " && cat valgrind.log | grep "All heap blocks were freed" | wc -l
echo -n "Good Valgrind Runs: fds   : " && cat valgrind.log | grep "FILE DESCRIPTORS: 0 open (0 std) at exit" | wc -l
mv valgrind.log ../pipex_testing

Good Valgrind Runs: Memory: 21
Good Valgrind Runs: fds   : 21
